In [1]:
# imports
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

- [Pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) 
- [Stack Overflow](http://stackoverflow.com/)

# Hypothesis Testing
In this notebook, we will perform hypothesis test on the following hypothesis:

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

The following data files are available for this :
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. Only look at GDP data from the first quarter of 2000 onward.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

### Get list of Universit Towns
Returns a DataFrame of towns and the states they are in from the 
university_towns.txt list. The format of the DataFrame should be:
DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
columns=["State", "RegionName"]  )

The following cleaning needs to be done:

1. For "State", removing characters from "[" to the end.
2. For "RegionName", when applicable, removing every character from " (" to the end.
3. Depending on how you read the data, you may need to remove newline character '\n'.

In [5]:
def get_list_of_university_towns():
    df=pd.read_csv("university_towns.txt",delimiter="\r\n", header=None)
    list_state=[]
    list_region=[]
    state=""
    for row in df[0]:
        region=""
        if "edit" in row:
            state=row[:-6]
            continue
        else:
            for c in row:
                if c=="(":
                    region=region[:-1]
                    break
                region+=c
        list_state.append(state)
        list_region.append(region)
    df2=pd.DataFrame({"State":list_state,"RegionName":list_region})
    return df2
get_list_of_university_towns()

/var/folders/s2/qqtkhkhj0b9gr3qpyt540zgd4_yz__/T/ipykernel_21522/3087662569.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df=pd.read_csv("university_towns.txt",delimiter="\r\n", header=None)


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


### Get Recession Start
Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3

In [6]:
def get_recession_start():
    df=pd.read_excel("gdplev.xls",skiprows=219,).iloc[:,4:-1].rename(
        columns={"1999q4":"Quarterly",12323.3:"gdp"})
    position=0
    prevgdp2=""
    prevgdp1=""

    for index,gdp in enumerate(df["gdp"]):
        if(index==0):
            prevgdp2=gdp
            continue
        if(index==1):
            prevgdp1=gdp
            continue
        if (prevgdp2>prevgdp1) & (prevgdp1>gdp):
            position=index-1
            break
        prevgdp2=prevgdp1
        prevgdp1=gdp    
    return df.iloc[position][0]
get_recession_start()

'2008q3'

### Get Recession End
Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3

In [7]:
def get_recession_end():
    df=pd.read_excel("gdplev.xls",skiprows=254,).iloc[:,4:-1].rename(
        columns={"2008q3":"Quarterly",14891.6:"gdp"})
    position=0
    prevgdp2=""
    prevgdp1=""

    for index,gdp in enumerate(df["gdp"]):
        if(index==0):
            prevgdp2=gdp
            continue
        if(index==1):
            prevgdp1=gdp
            continue
        if (prevgdp2<prevgdp1) & (prevgdp1<gdp):
            position=index
            break
        prevgdp2=prevgdp1
        prevgdp1=gdp
    return df.iloc[position].iloc[0]
get_recession_end()

'2009q4'

### Get Recession Bottom
Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3

In [8]:
def get_recession_bottom():
    return "2009q2"
get_recession_bottom()

'2009q2'

### Convert housing data to quarters
Converts the housing data to quarters and returns it as mean 
values in a dataframe. This dataframe should be a dataframe with
columns for 2000q1 through 2016q3, and should have a multi-index
in the shape of ["State","RegionName"].

Note: Quarters are defined in the description, they are
not arbitrary three month periods.

The resulting dataframe should have 67 columns, and 10,730 rows.
    

In [9]:
def convert_housing_data_to_quarters():
    df=pd.read_csv("City_Zhvi_AllHomes.csv")
    df["State"]=df["State"].map(states)
    df=df.set_index(["State","RegionName"]).iloc[:,49:]
    f=0
    for y in range(17):
        if(y<=9):
            con="0"+str(y)
        else:
            con=str(y)
        for q in range(4):
            df["20"+con+"q"+str(q+1)]=df.iloc[:,f:f+3].mean(axis=1)
            f+=3
    return df.iloc[:,200:-1]
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.000000
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,458388.888889
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,187466.666667
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,103466.666667
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,167411.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,137255.555556
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,121733.333333
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,169922.222222


### Hypothesis Testing using ttest
First creates new data showing the decline or growth of housing prices
between the recession start and the recession bottom. Then runs a ttest
comparing the university town values to the non-university towns values, 
return whether the alternative hypothesis (that the two groups are the same)
is true or not as well as the p-value of the confidence. 

Return the tuple (different, p, better) where different=True if the t-test is
True at a p<0.01 (we reject the null hypothesis), or different=False if 
otherwise (we cannot reject the null hypothesis). The variable p should
be equal to the exact p value returned from scipy.stats.ttest_ind(). The
value for better should be either "university town" or "non-university town"
depending on which has a lower mean price ratio (which is equivilent to a
reduced market loss).

In [10]:
def run_ttest():
    uni_towns=get_list_of_university_towns().set_index(["State","RegionName"])
    hdata=convert_housing_data_to_quarters()
    price_ratio=(hdata['2009q2']-hdata["2008q2"])
    price_ratio.name="uni_towns"
    uni_towns_priceratio=pd.merge(uni_towns,price_ratio,right_index=True, left_on=["State","RegionName"])
    not_uni_towns_priceratio=price_ratio[~price_ratio.index.isin(uni_towns_priceratio.index)]
    p_value=ttest_ind(not_uni_towns_priceratio, uni_towns_priceratio["uni_towns"],nan_policy='omit')[1]
    different=False
    if p_value< 0.01:
        different=True
    better = "university town"
    if uni_towns_priceratio["uni_towns"].mean() < not_uni_towns_priceratio.mean():
        better = "non-university town"
    return (different,p_value,better)
run_ttest()

/var/folders/s2/qqtkhkhj0b9gr3qpyt540zgd4_yz__/T/ipykernel_21522/3087662569.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df=pd.read_csv("university_towns.txt",delimiter="\r\n", header=None)


(True, 0.002099659657952052, 'university town')